In [1]:
import pandas as pd

In [2]:
cal = pd.read_csv('calendar.csv')
list_1 = pd.read_csv('listings.csv')
rev = pd.read_csv('reviews.csv')
neigh = pd.read_csv('neigh.csv')

In [3]:
cal['available'] = cal['available'].replace('f',False).replace('t',True)

In [4]:
def clean_strs(series):
    series = series.str.strip('$').str.replace(',','')
    return series.astype(float)

In [5]:
cal['price'] = clean_strs(cal['price'])
cal['adjusted_price'] = clean_strs(cal['adjusted_price'])

In [6]:
cal.to_csv('cal.csv',index=False,header=False)

In [7]:
REMOVE_COLS = ['scrape_id','last_scraped','smart_location','calendar_last_scraped']

In [8]:
HOST_COLS = ['host_id','host_url','host_name','host_since',
'host_location','host_about','host_response_time','host_response_rate',
'host_acceptance_rate','host_is_superhost','host_thumbnail_url',
'host_picture_url','host_neighbourhood','host_listings_count',
'host_total_listings_count','host_verifications',
'host_has_profile_pic','host_identity_verified']

In [10]:
listing = list_1.drop(REMOVE_COLS,axis=1)

In [11]:
listing = listing.drop(HOST_COLS,axis=1)

In [12]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [13]:
listing.rename(columns={'listing_url':'url'},inplace=True)

In [14]:
listing['zipcode'] = listing['zipcode'].fillna(0).astype(int)

In [15]:
def series_to_bool(series):
    series = series.replace('t',True).replace('f',False)
    return series

In [16]:
listing['is_location_exact'] = series_to_bool(listing['is_location_exact'])
listing['require_guest_profile_picture'] = series_to_bool(listing['require_guest_profile_picture'])
listing['require_guest_phone_verification'] = series_to_bool(listing['require_guest_phone_verification'])
listing['is_business_travel_ready'] = series_to_bool(listing['is_business_travel_ready'])
listing['has_availability'] = series_to_bool(listing['has_availability'])
listing['requires_license'] = series_to_bool(listing['requires_license'])
listing['instant_bookable'] = series_to_bool(listing['instant_bookable'])

In [17]:
listing['price'] = clean_strs(listing['price'])
listing['weekly_price'] = clean_strs(listing['weekly_price'])
listing['monthly_price'] = clean_strs(listing['monthly_price'])
listing['security_deposit'] = clean_strs(listing['security_deposit'])
listing['extra_people'] = clean_strs(listing['extra_people'])

In [18]:
listing['cleaning_fee'] = listing['cleaning_fee'].str.strip('$').astype(float)

In [19]:
listing.drop('license',axis=1,inplace=True)
listing.drop('jurisdiction_names',axis=1,inplace=True)

In [20]:
listing['neighbourhood'].fillna(0)
listing['neighbourhood_group_cleansed'].fillna(0)
listing['square_feet'].fillna(0)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2214    0.0
2215    0.0
2216    0.0
2217    0.0
2218    0.0
Name: square_feet, Length: 2219, dtype: float64

In [21]:
import numpy as np

In [22]:
for col in listing.columns:
    if 'url' in col:
        listing[col] = listing[str(col)].astype(str)

In [23]:
for col in listing.columns:
    if listing[str(col)].dtype == 'object':
        listing[col].fillna('',inplace=True)
        
    else:
        listing[str(col)].fillna(0,inplace=True)

In [55]:
sql = """INSERT INTO listing (id,
                        url,
                        name,
                        summary,
                        space,
                        description,
                        experiences_offered,
                        neighborhood_overview,
                        notes,
                        transit,
                        access,
                        interaction,
                        house_rules,
                        thumbnail_url,
                        medium_url,
                        picture_url,
                        xl_picture_url,
                        street,
                        neighbourhood,
                        neighbourhood_cleansed,
                        neighbourhood_group_cleansed,
                        city,
                        state,
                        zipcode,
                        market,
                        country_code,
                        country,
                        latitude,
                        longitude,
                        is_location_exact,
                        property_type,
                        room_type,
                        accommodates,
                        bathrooms,
                        bedrooms,
                        beds,
                        bed_type,
                        amenities,
                        square_feet,
                        price,
                        weekly_price,
                        monthly_price,
                        security_deposit,
                        cleaning_fee,
                        guests_included,
                        extra_people,
                        minimum_nights,
                        maximum_nights,
                        minimum_minimum_nights,
                        maximum_minimum_nights,
                        minimum_maximum_nights,
                        maximum_maximum_nights,
                        minimum_nights_avg_ntm,
                        maximum_nights_avg_ntm,
                        calendar_updated,
                        has_availability,
                        availability_30,
                        availability_60,
                        availability_90,
                        availability_365,
                        number_of_reviews,
                        number_of_reviews_ltm,
                        first_review,
                        last_review,
                        review_scores_rating,
                        review_scores_accuracy,
                        review_scores_cleanliness,
                        review_scores_checkin,
                        review_scores_communication,
                        review_scores_location,
                        review_scores_value,
                        requires_license,
                        instant_bookable,
                        is_business_travel_ready,
                        cancellation_policy,
                        require_guest_profile_picture,
                        require_guest_phone_verification,
                        calculated_host_listings_count,
                        calculated_host_listings_count_entire_homes,
                        calculated_host_listings_count_private_rooms,
                        calculated_host_listings_count_shared_rooms,
                        reviews_per_month)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

In [61]:
master_list = []
for row in listing.itertuples(index=False):
    master_list.append(row)
    cursor.execute(sql, row)
    connection.commit()
